# Metadata

**L1 Taxonomy** - SQL in Python

**L2 Taxonomy** - Views

**Subtopic** - Understanding Flask

**Use Case** - Implement a Flask application that validates incoming JSON payloads for a product catalog using custom request validation logic with Flask-WTF and WTForms.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
Flask==2.3.3
Flask-WTF==1.2.0
WTForms==3.1.0
Flask-Babel==4.0.0
```


# Prompt

A product catalog needs validation for different markets with varying business rules. Implement a Flask application using Flask-WTF forms that validates JSON product data with locale-specific rules, stores valid products in SQLite database, and returns internationalized error messages. Support US and German markets with different price ranges, categories, and certification requirements.

**Input Format**

The function receives JSON payload and locale parameter:

```python
validate_and_store_product(product_json: str, locale: str) -> dict
```

JSON payload structure:
```json
{
    "name": "string (1-100 chars, required)",
    "price": "string (decimal format, market-specific range)",
    "category": "string (must exist in market's category list)",
    "launch_date": "string (market-specific date format)",
    "certification": "string (required for de_DE market only)"
}
```

**Output Format**

Success Response:
```python
{
    'status': 'success',
    'product_id': 1,
    'errors': {},
    'message': 'Product saved successfully'
}
```

Validation Error Response:
```python
{
    'status': 'validation_error',
    'product_id': None,
    'errors': {
        'price': ['Price must be between 10.00 and 5,000.00'],
        'category': ['Category must be one of: Electronics, Books, Clothing']
    },
    'message': 'Validation failed'
}
```

Storage Error Response:
```python
{
    'status': 'storage_error',
    'product_id': None,
    'errors': {},
    'message': 'Database operation failed'
}
```

**Examples**

**Example 1 (Valid US Product):**

Input:
```python
product_json = '{"name": "Python Programming Book", "price": "29.99", "category": "Books", "launch_date": "03/15/2024"}'
locale = "en_US"
```

Output:
```python
{
    'status': 'success',
    'product_id': 1,
    'errors': {},
    'message': 'Product saved successfully'
}
```

**Example 2 (Missing German Certification):**

Input:
```python
product_json = '{"name": "Smartphone", "price": "599.99", "category": "Elektronik", "launch_date": "01.06.2024"}'
locale = "de_DE"
```

Output:
```python
{
    'status': 'validation_error',
    'product_id': None,
    'errors': {
        'certification': ['Zertifizierung ist erforderlich']
    },
    'message': 'Validierung fehlgeschlagen'
}
```

**Function Signature**

```python
def validate_and_store_product(product_json: str, locale: str) -> dict:
    """
    Validates product JSON using Flask-WTF and stores valid products in SQLite.
    
    Args:
        product_json: JSON string containing product data
        locale: Either 'en_US' or 'de_DE'
    
    Returns:
        dict: Validation result with status, product_id, errors, and message
    """

# Translation dictionary for error messages
MESSAGES = {
    'en_US': {
        'success': 'Product saved successfully',
        'validation_failed': 'Validation failed',
        'cert_required': 'Certification required'
    },
    'de_DE': {
        'success': 'Produkt erfolgreich gespeichert',
        'validation_failed': 'Validierung fehlgeschlagen',
        'cert_required': 'Zertifizierung ist erforderlich'
    }
}

# Mock external service (to be mocked in implementation)
def validate_certification(cert_code: str) -> bool:
    """External certification validation service"""
    pass
```



# Requirements

**Explicit Requirements:**
* Implement `validate_and_store_product()` function that processes JSON and locale parameters
* Create Flask-WTF form class that adapts validation rules based on locale ('en_US' or 'de_DE')
* Use in-memory SQLite database with products and categories tables for data storage
* Mock external certification validation service using `validate_certification()` function
* Return structured response dict with status, product_id, errors, and localized message fields
* Handle JSON parsing errors and return appropriate error responses with 'storage_error' status
* US Market: Price range 10.00-5,000.00, categories [Electronics, Books, Clothing], date format MM/DD/YYYY
* German Market: Price range 5.00-4,000.00, categories [Elektronik, Bücher, Kleidung], date format DD.MM.YYYY, certification required

**Implicit Requirements:**
* Price validation must check locale-specific decimal ranges using custom Flask-WTF validators
* Date format validation must use locale-specific patterns and reject invalid formats
* Category validation must check against locale-specific category lists stored in database
* German certification codes must pass external validation service (mock returns True for codes starting with 'CE')
* Error messages must be returned in appropriate language: English for en_US, German for de_DE
* Database schema must include locale field to support multi-market product storage
* Form validation must happen within Flask application context for proper WTF functionality
* All price values must be validated as valid decimal strings before conversion
* Missing required fields must generate field-specific error messages in correct language
* Invalid JSON input must return 'storage_error' status with appropriate error message

In [ ]:
# code

from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, DecimalField
from wtforms.validators import DataRequired, Length, ValidationError
import json
from datetime import datetime

app = Flask(__name__)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///:memory:"
app.config["SECRET_KEY"] = "secret"
app.config["WTF_CSRF_ENABLED"] = False
db = SQLAlchemy(app)

MESSAGES = {
    "en_US": {
        "success": "Product saved successfully",
        "validation_failed": "Validation failed",
        "cert_required": "Certification required",
    },
    "de_DE": {
        "success": "Produkt erfolgreich gespeichert",
        "validation_failed": "Validierung fehlgeschlagen",
        "cert_required": "Zertifizierung ist erforderlich",
    },
}


class Product(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    price = db.Column(db.Float, nullable=False)
    category = db.Column(db.String(100), nullable=False)
    launch_date = db.Column(db.Date, nullable=False)
    certification = db.Column(db.String(100))
    locale = db.Column(db.String(5), nullable=False)


class Category(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    locale = db.Column(db.String(5), nullable=False)


with app.app_context():
    db.create_all()
    db.session.add_all(
        [
            Category(name="Electronics", locale="en_US"),
            Category(name="Books", locale="en_US"),
            Category(name="Clothing", locale="en_US"),
            Category(name="Elektronik", locale="de_DE"),
            Category(name="Bücher", locale="de_DE"),
            Category(name="Kleidung", locale="de_DE"),
        ]
    )
    db.session.commit()


def validate_certification(cert_code: str) -> bool:
    return cert_code.startswith("CE")


def create_price_validator(min_val: float, max_val: float, message: str):
    def price_validator(form, field):
        try:
            value = float(field.data)
            if not (min_val <= value <= max_val):
                raise ValidationError(message)
        except (ValueError, TypeError):
            raise ValidationError(message)

    return price_validator


def create_category_validator(locale: str, message: str):
    def category_validator(form, field):
        with app.app_context():
            valid_categories = [
                cat.name
                for cat in Category.query.filter_by(locale=locale).all()
            ]
            if field.data not in valid_categories:
                raise ValidationError(message)

    return category_validator


def create_date_validator(date_format: str, message: str):
    def date_validator(form, field):
        try:
            datetime.strptime(field.data, date_format)
        except ValueError:
            raise ValidationError(message)

    return date_validator


class ProductFormUS(FlaskForm):
    name = StringField(
        "Name", validators=[DataRequired(), Length(min=1, max=100)]
    )
    price = DecimalField("Price", validators=[DataRequired()])
    category = StringField("Category", validators=[DataRequired()])
    launch_date = StringField("Launch Date", validators=[DataRequired()])

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.price.validators.append(
            create_price_validator(
                10.00, 5000.00, "Price must be between 10.00 and 5,000.00"
            )
        )
        self.category.validators.append(
            create_category_validator(
                "en_US",
                "Category must be one of: Electronics, Books, Clothing",
            )
        )
        self.launch_date.validators.append(
            create_date_validator("%m/%d/%Y", "Date format must be MM/DD/YYYY")
        )


class ProductFormDE(FlaskForm):
    name = StringField(
        "Name", validators=[DataRequired(), Length(min=1, max=100)]
    )
    price = DecimalField("Price", validators=[DataRequired()])
    category = StringField("Category", validators=[DataRequired()])
    launch_date = StringField("Launch Date", validators=[DataRequired()])
    certification = StringField(
        "Certification",
        validators=[DataRequired(message="Zertifizierung ist erforderlich")],
    )

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.price.validators.append(
            create_price_validator(
                5.00, 4000.00, "Preis muss zwischen 5.00 und 4.000,00 liegen"
            )
        )
        self.category.validators.append(
            create_category_validator(
                "de_DE",
                "Kategorie muss eine der folgenden sein: Elektronik, "
                "Bücher, Kleidung",
            )
        )
        self.launch_date.validators.append(
            create_date_validator(
                "%d.%m.%Y", "Datumsformat muss DD.MM.YYYY sein"
            )
        )

    def validate_certification(self, field):
        if not validate_certification(field.data):
            raise ValidationError(MESSAGES["de_DE"]["cert_required"])


def validate_and_store_product(product_json: str, locale: str) -> dict:
    try:
        product_data = json.loads(product_json)
    except (json.JSONDecodeError, TypeError):
        return {
            "status": "storage_error",
            "product_id": None,
            "errors": {},
            "message": "Database operation failed",
        }

    if locale not in ["en_US", "de_DE"]:
        return {
            "status": "storage_error",
            "product_id": None,
            "errors": {},
            "message": "Database operation failed",
        }

    with app.app_context():
        if locale == "en_US":
            form = ProductFormUS(data=product_data)
        else:
            form = ProductFormDE(data=product_data)

        if form.validate():
            if locale == "en_US":
                launch_date = datetime.strptime(
                    product_data["launch_date"], "%m/%d/%Y"
                ).date()
            else:
                launch_date = datetime.strptime(
                    product_data["launch_date"], "%d.%m.%Y"
                ).date()

            product = Product(
                name=form.name.data,
                price=float(form.price.data),
                category=form.category.data,
                launch_date=launch_date,
                certification=product_data.get("certification"),
                locale=locale,
            )

            try:
                db.session.add(product)
                db.session.commit()
                return {
                    "status": "success",
                    "product_id": product.id,
                    "errors": {},
                    "message": MESSAGES[locale]["success"],
                }
            except Exception:
                db.session.rollback()
                return {
                    "status": "storage_error",
                    "product_id": None,
                    "errors": {},
                    "message": "Database operation failed",
                }
        else:
            errors = {}
            for field_name, field_errors in form.errors.items():
                if isinstance(field_errors, list):
                    errors[field_name] = field_errors
                else:
                    errors[field_name] = [field_errors]

            return {
                "status": "validation_error",
                "product_id": None,
                "errors": errors,
                "message": MESSAGES[locale]["validation_failed"],
            }


In [ ]:
# tests

"""Module consists test cases to validate 'validate_and_store_product'."""

import unittest
from main import app, db, validate_and_store_product, Category
import json


class TestProductValidation(unittest.TestCase):
    """Unit tests for validate_and_store_product function."""

    def setUp(self):
        """Set up test fixtures before each test method."""
        self.app = app
        self.app_context = self.app.app_context()
        self.app_context.push()

        db.drop_all()
        db.create_all()

        db.session.add_all([
            Category(name="Electronics", locale="en_US"),
            Category(name="Books", locale="en_US"),
            Category(name="Clothing", locale="en_US"),
            Category(name="Elektronik", locale="de_DE"),
            Category(name="Bücher", locale="de_DE"),
            Category(name="Kleidung", locale="de_DE"),
        ])
        db.session.commit()

    def tearDown(self):
        """Clean up after each test method."""
        db.session.remove()
        self.app_context.pop()

    def test_valid_us_product(self):
        """Test valid product submission for US market."""
        payload = {
            "name": "USB Cable",
            "price": "15.99",
            "category": "Electronics",
            "launch_date": "08/20/2024"
        }
        result = validate_and_store_product(json.dumps(payload), "en_US")
        self.assertEqual(result["status"], "success")

    def test_valid_german_product_with_cert(self):
        """Test valid product with CE cert for German market."""
        payload = {
            "name": "Router",
            "price": "199.99",
            "category": "Elektronik",
            "launch_date": "15.07.2024",
            "certification": "CE-1234"
        }
        result = validate_and_store_product(json.dumps(payload), "de_DE")
        self.assertEqual(result["status"], "success")

    def test_missing_certification_german(self):
        """Test missing certification for German product."""
        payload = {
            "name": "Smartphone",
            "price": "599.99",
            "category": "Elektronik",
            "launch_date": "01.06.2024"
        }
        result = validate_and_store_product(json.dumps(payload), "de_DE")
        self.assertEqual(result["status"], "validation_error")
        self.assertIn("certification", result["errors"])

    def test_invalid_price_us(self):
        """Test US product with price below allowed range."""
        payload = {
            "name": "Book",
            "price": "5.00",
            "category": "Books",
            "launch_date": "03/10/2024"
        }
        result = validate_and_store_product(json.dumps(payload), "en_US")
        self.assertEqual(result["status"], "validation_error")
        self.assertIn("price", result["errors"])

    def test_invalid_price_de(self):
        """Test German product with price above allowed range."""
        payload = {
            "name": "TV",
            "price": "4999.00",
            "category": "Elektronik",
            "launch_date": "10.06.2024",
            "certification": "CE-0001"
        }
        result = validate_and_store_product(json.dumps(payload), "de_DE")
        self.assertEqual(result["status"], "validation_error")
        self.assertIn("price", result["errors"])

    def test_invalid_category_us(self):
        """Test invalid category for US product."""
        payload = {
            "name": "Gadget",
            "price": "29.99",
            "category": "Gadgets",
            "launch_date": "02/12/2024"
        }
        result = validate_and_store_product(json.dumps(payload), "en_US")
        self.assertEqual(result["status"], "validation_error")
        self.assertIn("category", result["errors"])

    def test_invalid_category_de(self):
        """Test invalid category for German product."""
        payload = {
            "name": "Camera",
            "price": "299.00",
            "category": "Fotografie",
            "launch_date": "01.05.2024",
            "certification": "CE-9000"
        }
        result = validate_and_store_product(json.dumps(payload), "de_DE")
        self.assertEqual(result["status"], "validation_error")
        self.assertIn("category", result["errors"])

    def test_invalid_date_format_us(self):
        """Test invalid US date format."""
        payload = {
            "name": "Laptop",
            "price": "999.99",
            "category": "Electronics",
            "launch_date": "2024-03-10"
        }
        result = validate_and_store_product(json.dumps(payload), "en_US")
        self.assertEqual(result["status"], "validation_error")
        self.assertIn("launch_date", result["errors"])

    def test_invalid_date_format_de(self):
        """Test invalid German date format."""
        payload = {
            "name": "Tablet",
            "price": "450.00",
            "category": "Elektronik",
            "launch_date": "2024/06/01",
            "certification": "CE-1111"
        }
        result = validate_and_store_product(json.dumps(payload), "de_DE")
        self.assertEqual(result["status"], "validation_error")
        self.assertIn("launch_date", result["errors"])

    def test_invalid_json_input(self):
        """Test malformed JSON input handling."""
        bad_json = '{"name": "Speaker", "price": "99.99",}'
        result = validate_and_store_product(bad_json, "en_US")
        self.assertEqual(result["status"], "storage_error")

    def test_missing_required_field(self):
        """Test product missing required name field."""
        payload = {
            "price": "25.00",
            "category": "Books",
            "launch_date": "04/01/2024"
        }
        result = validate_and_store_product(json.dumps(payload), "en_US")
        self.assertEqual(result["status"], "validation_error")
        self.assertIn("name", result["errors"])

    def test_invalid_price_format(self):
        """Test product with invalid price string."""
        payload = {
            "name": "Charger",
            "price": "twenty",
            "category": "Electronics",
            "launch_date": "06/10/2024"
        }
        result = validate_and_store_product(json.dumps(payload), "en_US")
        self.assertEqual(result["status"], "validation_error")
        self.assertIn("price", result["errors"])

    def test_certification_invalid_code(self):
        """Test German product with invalid certification code."""
        payload = {
            "name": "Scanner",
            "price": "300.00",
            "category": "Elektronik",
            "launch_date": "12.12.2024",
            "certification": "X-0001"
        }
        result = validate_and_store_product(json.dumps(payload), "de_DE")
        self.assertEqual(result["status"], "validation_error")
        self.assertIn("certification", result["errors"])

    def test_name_length_too_long(self):
        """Test product name exceeding 100 characters."""
        payload = {
            "name": "x" * 101,
            "price": "30.00",
            "category": "Books",
            "launch_date": "03/03/2024"
        }
        result = validate_and_store_product(json.dumps(payload), "en_US")
        self.assertEqual(result["status"], "validation_error")
        self.assertIn("name", result["errors"])

    def test_name_minimum_length(self):
        """Test product name at minimum length."""
        payload = {
            "name": "X",
            "price": "12.00",
            "category": "Books",
            "launch_date": "04/12/2024"
        }
        result = validate_and_store_product(json.dumps(payload), "en_US")
        self.assertEqual(result["status"], "success")


# Model Breaking Proof

#### Model Breaking Task URL: <https://rlhf-v3.turing.com/prompt/d115248b-b3fa-4dd5-833d-1d0a9692780a?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F67929%2Fview>

#### Model code:

```python
# code generated by the model

from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, DecimalField
from wtforms.validators import DataRequired, Length, NumberRange, ValidationError
import json
from datetime import datetime

# Initialize Flask app and database
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///:memory:'
app.config['SECRET_KEY'] = 'secret'
db = SQLAlchemy(app)

# Constants
# <Issue>: Extra keys not in provided MESSAGES dict (price_range, category_list, date_format, parsing_error)
MESSAGES = {
    'en_US': {
        'success': 'Product saved successfully',
        'validation_failed': 'Validation failed',
        'price_range': 'Price must be between 10.00 and 5,000.00',
        'category_list': 'Category must be one of: Electronics, Books, Clothing',
        'date_format': 'Date format should be MM/DD/YYYY',
        'cert_required': 'Certification required',
        'parsing_error': 'Error parsing JSON'
    },
    'de_DE': {
        'success': 'Produkt erfolgreich gespeichert',
        'validation_failed': 'Validierung fehlgeschlagen',
        'price_range': 'Preis muss zwischen 5,00 und 4,000.00 liegen',
        'category_list': 'Kategorie muss eine der folgenden sein: Elektronik, Bücher, Kleidung',
        'date_format': 'Datum sollte im Format DD.MM.YYYY sein',
        'cert_required': 'Zertifizierung ist erforderlich',
        'parsing_error': 'Fehler beim Parsen von JSON'
    }
}

# Database models
class Product(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    price = db.Column(db.Float, nullable=False)
    category = db.Column(db.String(100), nullable=False)
    launch_date = db.Column(db.Date, nullable=False)
    certification = db.Column(db.String(100))
    locale = db.Column(db.String(5), nullable=False)

# <Issue>: Category table missing locale column for market-specific filtering
class Category(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)

# Populate categories (in-memory, just for demonstration)
with app.app_context():
    db.create_all()
    db.session.add_all([
        Category(name="Electronics"),
        Category(name="Books"),
        Category(name="Clothing"),
        Category(name="Elektronik"),
        Category(name="Bücher"),
        Category(name="Kleidung"),
    ])
    db.session.commit()

# Mock external certification validation service
def validate_certification(cert_code: str) -> bool:
    return cert_code.startswith('CE')

# Define WTForms for validation
class ProductFormUS(FlaskForm):
    name = StringField('Name', validators=[DataRequired(), Length(min=1, max=100)])
    price = DecimalField('Price', validators=[DataRequired(), NumberRange(min=10.00, max=5000.00)])
    category = StringField('Category', validators=[DataRequired()])
    launch_date = StringField('Launch Date', validators=[DataRequired()])

    def validate_category(form, field):
        # <Issue>: Hardcoded categories instead of querying database with locale filter
        valid_categories = ['Electronics', 'Books', 'Clothing']
        if field.data not in valid_categories:
            raise ValidationError(MESSAGES['en_US']['category_list'])

    def validate_launch_date(form, field):
        try:
            datetime.strptime(field.data, '%m/%d/%Y')
        except ValueError:
            raise ValidationError(MESSAGES['en_US']['date_format'])

class ProductFormDE(FlaskForm):
    name = StringField('Name', validators=[DataRequired(), Length(min=1, max=100)])
    price = DecimalField('Price', validators=[DataRequired(), NumberRange(min=5.00, max=4000.00)])
    category = StringField('Category', validators=[DataRequired()])
    launch_date = StringField('Launch Date', validators=[DataRequired()])
    certification = StringField('Certification', validators=[DataRequired()])

    def validate_category(form, field):
        # <Issue>: Hardcoded categories instead of querying database with locale filter
        valid_categories = ['Elektronik', 'Bücher', 'Kleidung']
        if field.data not in valid_categories:
            raise ValidationError(MESSAGES['de_DE']['category_list'])

    def validate_launch_date(form, field):
        try:
            datetime.strptime(field.data, '%d.%m.%Y')
        except ValueError:
            raise ValidationError(MESSAGES['de_DE']['date_format'])

    def validate_certification(form, field):
        if not validate_certification(field.data):
            raise ValidationError(MESSAGES['de_DE']['cert_required'])

# Validation and storage function
def validate_and_store_product(product_json: str, locale: str) -> dict:
    try:
        product_data = json.loads(product_json)
    except json.JSONDecodeError:
        return {
            'status': 'storage_error',
            'product_id': None,
            'errors': {},
            # <Issue>: Should return 'Database operation failed', not locale-specific message
            # <Issue>: No locale validation - MESSAGES[locale] can fail if locale is invalid
            'message': MESSAGES[locale]['parsing_error']
        }

    with app.app_context():  # Ensure we have app context for database operations
        if locale == 'en_US':
            # <Issue>: Missing meta={'csrf': False} for JSON validation
            form = ProductFormUS(data=product_data)
        elif locale == 'de_DE':
            # <Issue>: Missing meta={'csrf': False} for JSON validation
            form = ProductFormDE(data=product_data)
        else:
            return {
                'status': 'storage_error',
                'product_id': None,
                'errors': {},
                'message': 'Unsupported locale'
            }

        if form.validate():
            # Convert launch_date to proper format for storage
            if locale == 'en_US':
                formatted_date = datetime.strptime(product_data['launch_date'], '%m/%d/%Y').date()
            elif locale == 'de_DE':
                formatted_date = datetime.strptime(product_data['launch_date'], '%d.%m.%Y').date()
            
            # Store product
            product = Product(
                name=form.name.data,
                price=float(form.price.data),
                category=form.category.data,
                launch_date=formatted_date,
                # <Issue>: form.certification doesn't exist on ProductFormUS
                certification=form.certification.data if 'certification' in product_data else None,
                locale=locale
            )
            db.session.add(product)
            db.session.commit()
            
            return {
                'status': 'success',
                'product_id': product.id,
                'errors': {},
                'message': MESSAGES[locale]['success']
            }
        else:
            # <Issue>: Returns {field: err} instead of {field: [err]} - errors must be arrays
            errors = {field: err for field, err in form.errors.items()}
            return {
                'status': 'validation_error',
                'product_id': None,
                'errors': errors,
                'message': MESSAGES[locale]['validation_failed']
            }

# Example usage within Flask app context
with app.app_context():
    response = validate_and_store_product(
        '{"name": "Smartphone", "price": "599.99", "category": "Elektronik", "launch_date": "01.06.2024", "certification": "CE123"}',
        'de_DE'
    )
    print(response)

# Please note this example is for script-based execution to demonstrate functionality. To run as a proper flask server, further setup is necessary.
```


